## Loading and Merging Data from CSV files

In [16]:
import pandas as pd

# List of file names for March, April, and May across four years
file_names = [
    'Los_Angeles_January_2024.csv','Los_Angeles_February_2024.csv','Los_Angeles_March_2024.csv',
    'Los_Angeles_April_2024.csv','Los_Angeles_May_2024.csv','Los_Angeles_June_2024.csv'
]

# List to store dataframes
dataframes = []

# Read each CSV file and append to the list
for file_name in file_names:
    df = pd.read_csv(file_name)
    dataframes.append(df)

# Check if dataframes list is not empty
if dataframes:
    # Concatenate all dataframes into a single dataframe
    merged_LA_df = pd.concat(dataframes, ignore_index=True)
# Save the merged and processed DataFrame to a CSV file
merged_LA_df.to_csv('LA_merged_2020_2023.csv', index=False)  #optional

## Data Inspection

In [17]:
# Display the first few rows of the DataFrame to get a quick look at the data
print(merged_LA_df.head())

   location_id         location_name parameter  value unit  \
0         7936  Los Angeles - N. Mai        co    0.3  ppm   
1         7936  Los Angeles - N. Mai        co    0.3  ppm   
2         7936  Los Angeles - N. Mai        co    0.5  ppm   
3         7936  Los Angeles - N. Mai        co    0.6  ppm   
4         7936  Los Angeles - N. Mai        co    0.8  ppm   

                 datetimeUtc              datetimeLocal             timezone  \
0  2024-01-01T01:00:00+00:00  2023-12-31T17:00:00-08:00  America/Los_Angeles   
1  2024-01-01T02:00:00+00:00  2023-12-31T18:00:00-08:00  America/Los_Angeles   
2  2024-01-01T03:00:00+00:00  2023-12-31T19:00:00-08:00  America/Los_Angeles   
3  2024-01-01T04:00:00+00:00  2023-12-31T20:00:00-08:00  America/Los_Angeles   
4  2024-01-01T05:00:00+00:00  2023-12-31T21:00:00-08:00  America/Los_Angeles   

    latitude   longitude  country_iso  isMobile  isMonitor  \
0  34.066429 -118.226755          NaN       NaN        NaN   
1  34.066429 -118.2267

In [18]:
# Display a concise summary of the DataFrame
merged_LA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20067 entries, 0 to 20066
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location_id    20067 non-null  int64  
 1   location_name  20067 non-null  object 
 2   parameter      20067 non-null  object 
 3   value          20067 non-null  float64
 4   unit           20067 non-null  object 
 5   datetimeUtc    20067 non-null  object 
 6   datetimeLocal  20067 non-null  object 
 7   timezone       20067 non-null  object 
 8   latitude       20067 non-null  float64
 9   longitude      20067 non-null  float64
 10  country_iso    0 non-null      float64
 11  isMobile       0 non-null      float64
 12  isMonitor      0 non-null      float64
 13  owner_name     20067 non-null  object 
 14  provider       20067 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 2.3+ MB


## Data Cleaning

#Handling missing values

In [19]:
# Check for missing values in each column and count them
merged_LA_df.isnull().sum()

location_id          0
location_name        0
parameter            0
value                0
unit                 0
datetimeUtc          0
datetimeLocal        0
timezone             0
latitude             0
longitude            0
country_iso      20067
isMobile         20067
isMonitor        20067
owner_name           0
provider             0
dtype: int64

In [20]:
#Define the columns to drop empty columns from the DataFrame
columns_to_drop=['country_iso','isMobile','isMonitor']
# Drop the specified columns from the DataFrame
merged_LA_df.drop(columns=columns_to_drop,inplace=True)
# Display the first few rows of the DataFrame to get a quick look at the data
print(merged_LA_df.head())

   location_id         location_name parameter  value unit  \
0         7936  Los Angeles - N. Mai        co    0.3  ppm   
1         7936  Los Angeles - N. Mai        co    0.3  ppm   
2         7936  Los Angeles - N. Mai        co    0.5  ppm   
3         7936  Los Angeles - N. Mai        co    0.6  ppm   
4         7936  Los Angeles - N. Mai        co    0.8  ppm   

                 datetimeUtc              datetimeLocal             timezone  \
0  2024-01-01T01:00:00+00:00  2023-12-31T17:00:00-08:00  America/Los_Angeles   
1  2024-01-01T02:00:00+00:00  2023-12-31T18:00:00-08:00  America/Los_Angeles   
2  2024-01-01T03:00:00+00:00  2023-12-31T19:00:00-08:00  America/Los_Angeles   
3  2024-01-01T04:00:00+00:00  2023-12-31T20:00:00-08:00  America/Los_Angeles   
4  2024-01-01T05:00:00+00:00  2023-12-31T21:00:00-08:00  America/Los_Angeles   

    latitude   longitude                         owner_name provider  
0  34.066429 -118.226755  Unknown Governmental Organization   AirNow  
1  3

#Conversion of Unit for further analysis

In [21]:
# Molecular weights of the pollutants
molecular_weights = {
    'co': 28.01,  # Carbon Monoxide
    'no2': 46.0055,  # Nitrogen Dioxide
    'o3': 48.00,  # Ozone
    # PM10 and PM2.5 do not need conversion
}
# Conversion from ppm to µg/m³
def ppm_to_ugm3(concentration, molecular_weight):
    return concentration * molecular_weight * 1000 / 24.45

# Convert ppm to µg/m³ where necessary
def convert_units(row):
    if row['unit'] == 'ppm' and row['parameter'] in molecular_weights:
        return ppm_to_ugm3(row['value'], molecular_weights[row['parameter']])
    return row['value']

# Apply the conversion
merged_LA_df['value'] = merged_LA_df.apply(convert_units, axis=1)

# Now all values should be in µg/m³
merged_LA_df['unit'] = 'µg/m³'

print(merged_LA_df)

       location_id         location_name parameter       value   unit  \
0             7936  Los Angeles - N. Mai        co  343.680982  µg/m³   
1             7936  Los Angeles - N. Mai        co  343.680982  µg/m³   
2             7936  Los Angeles - N. Mai        co  572.801636  µg/m³   
3             7936  Los Angeles - N. Mai        co  687.361963  µg/m³   
4             7936  Los Angeles - N. Mai        co  916.482618  µg/m³   
...            ...                   ...       ...         ...    ...   
20062         7936  Los Angeles - N. Mai      pm25   23.000000  µg/m³   
20063         7936  Los Angeles - N. Mai      pm25   23.000000  µg/m³   
20064         7936  Los Angeles - N. Mai      pm25   15.000000  µg/m³   
20065         7936  Los Angeles - N. Mai      pm25   15.000000  µg/m³   
20066         7936  Los Angeles - N. Mai      pm25   12.000000  µg/m³   

                     datetimeUtc              datetimeLocal  \
0      2024-01-01T01:00:00+00:00  2023-12-31T17:00:00-08:00 

#Checking for Duplicate Values

In [22]:
#Check for duplicate rows in the dataset
merged_LA_df_dup=merged_LA_df.duplicated().sum()
#printing the value
merged_LA_df_dup

0

#Converting data types for further Analysis

In [23]:
# converting 'datetimeUtc' column in to datetime format
merged_LA_df['datetimeUtc'] = pd.to_datetime(merged_LA_df['datetimeUtc'])

## Data Manuplation

#Pivot the DataFrame to Have the Pollutants as Column

In [24]:
# Pivot the DataFrame with multi-level index including 'location_name' and 'unit'
merged_LA_df_pivot = pd.pivot_table(merged_LA_df, 
                                index=['datetimeUtc','datetimeLocal','location_name', 'unit'], 
                                columns='parameter', 
                                values='value', 
                                aggfunc='mean')

# Reset index to convert the multi-level index back into columns
merged_LA_df_pivot.reset_index(inplace=True)

# Reorder columns as desired
merged_LA_df_pivot = merged_LA_df_pivot[['location_name', 'datetimeUtc','datetimeLocal','co','no2','o3','pm10','pm25', 'unit']]

In [25]:
# Display a concise summary of the pivotted DataFrame
merged_LA_df_pivot.info() #presence of missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4125 entries, 0 to 4124
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   location_name  4125 non-null   object             
 1   datetimeUtc    4125 non-null   datetime64[ns, UTC]
 2   datetimeLocal  4125 non-null   object             
 3   co             4025 non-null   float64            
 4   no2            4043 non-null   float64            
 5   o3             4043 non-null   float64            
 6   pm10           3954 non-null   float64            
 7   pm25           4002 non-null   float64            
 8   unit           4125 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(5), object(3)
memory usage: 290.2+ KB


In [26]:
# Function to fill half NaNs with 0 and half with the preceding value
def balanced_fillna(series):
    # Get the indices of NaNs
    nan_indices = series[series.isna()].index
    n = len(nan_indices)
    # Split the indices into two halves
    half = n // 2
    # Fill the first half with 0
    series.iloc[nan_indices[:half]] = 0
    # Fill the second half with the preceding value
    series.iloc[nan_indices[half:]] = series.ffill().iloc[nan_indices[half:]]
    return series

# Apply the balanced fillna function to each pollutant column
for pollutant in ['co', 'no2', 'o3', 'pm10', 'pm25']:
    merged_LA_df_pivot[pollutant] = balanced_fillna(merged_LA_df_pivot[pollutant])

# Save the cleaned and converted data to a new CSV file
merged_LA_df_pivot.to_csv('LA_merged_2024_balanced.csv', index=False)

# Display the first few rows of the updated DataFrame for verification
print(merged_LA_df_pivot.head())

parameter         location_name               datetimeUtc  \
0          Los Angeles - N. Mai 2024-01-01 01:00:00+00:00   
1          Los Angeles - N. Mai 2024-01-01 02:00:00+00:00   
2          Los Angeles - N. Mai 2024-01-01 03:00:00+00:00   
3          Los Angeles - N. Mai 2024-01-01 04:00:00+00:00   
4          Los Angeles - N. Mai 2024-01-01 05:00:00+00:00   

parameter              datetimeLocal          co        no2         o3  pm10  \
0          2023-12-31T17:00:00-08:00  343.680982  13.171309  58.895706   7.0   
1          2023-12-31T18:00:00-08:00  343.680982  30.105849  43.190184  11.0   
2          2023-12-31T19:00:00-08:00  572.801636  39.513926  25.521472  12.0   
3          2023-12-31T20:00:00-08:00  687.361963  45.158773  15.705521  18.0   
4          2023-12-31T21:00:00-08:00  916.482618  54.566851   1.963190  33.0   

parameter  pm25   unit  
0           3.0  µg/m³  
1           8.5  µg/m³  
2           6.7  µg/m³  
3          14.0  µg/m³  
4          25.0  µg/m³  


C:\Users\Admin\AppData\Local\Temp\ipykernel_10152\2724284931.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices[half:]] = series.ffill().iloc[nan_indices[half:]]
C:\Users\Admin\AppData\Local\Temp\ipykernel_10152\2724284931.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices[:half]] = 0


In [27]:
# Display a concise summary of pivotted DataFrame for verification
merged_LA_df_pivot.info() #after filling missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4125 entries, 0 to 4124
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   location_name  4125 non-null   object             
 1   datetimeUtc    4125 non-null   datetime64[ns, UTC]
 2   datetimeLocal  4125 non-null   object             
 3   co             4125 non-null   float64            
 4   no2            4125 non-null   float64            
 5   o3             4125 non-null   float64            
 6   pm10           4125 non-null   float64            
 7   pm25           4125 non-null   float64            
 8   unit           4125 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(5), object(3)
memory usage: 290.2+ KB


## Exploratory Data Analysis

In [28]:
# List of columns to apply absolute value transformation
#believing that the negative values in these columns are due to measurement errors or noise, and the true values should be non-negative.
columns_to_transform = ['pm10', 'pm25']

# Apply absolute value transformation to each specified column
merged_LA_df_pivot[columns_to_transform] = merged_LA_df_pivot[columns_to_transform].abs()

In [29]:
# Summary statistics for numerical columns
print(merged_LA_df_pivot.describe())

parameter           co          no2           o3         pm10         pm25
count      4125.000000  4125.000000  4125.000000  4125.000000  4125.000000
mean        376.452178    25.870686    53.615795    20.328727    10.946691
std         213.448233    16.466243    32.788224    11.589866     6.570252
min           0.000000     0.000000     0.000000     0.000000     0.000000
25%         229.120654    12.606824    25.521472    12.000000     6.400000
50%         343.680982    20.697771    58.895706    20.000000     9.900000
75%         458.241309    37.632311    78.527607    27.000000    15.000000
max        1603.844581    84.672699   153.128834    82.000000    55.000000


In [30]:
merged_LA_df_pivot = merged_LA_df_pivot.rename(columns={'co': 'CO', 'no2': 'NO2','o3':'O3'})
print(merged_LA_df_pivot.head())

parameter         location_name               datetimeUtc  \
0          Los Angeles - N. Mai 2024-01-01 01:00:00+00:00   
1          Los Angeles - N. Mai 2024-01-01 02:00:00+00:00   
2          Los Angeles - N. Mai 2024-01-01 03:00:00+00:00   
3          Los Angeles - N. Mai 2024-01-01 04:00:00+00:00   
4          Los Angeles - N. Mai 2024-01-01 05:00:00+00:00   

parameter              datetimeLocal          CO        NO2         O3  pm10  \
0          2023-12-31T17:00:00-08:00  343.680982  13.171309  58.895706   7.0   
1          2023-12-31T18:00:00-08:00  343.680982  30.105849  43.190184  11.0   
2          2023-12-31T19:00:00-08:00  572.801636  39.513926  25.521472  12.0   
3          2023-12-31T20:00:00-08:00  687.361963  45.158773  15.705521  18.0   
4          2023-12-31T21:00:00-08:00  916.482618  54.566851   1.963190  33.0   

parameter  pm25   unit  
0           3.0  µg/m³  
1           8.5  µg/m³  
2           6.7  µg/m³  
3          14.0  µg/m³  
4          25.0  µg/m³  


In [ ]:
# Define the AQI breakpoints for each pollutant
aqi_breakpoints = {
    'pm25': [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 350.4, 301, 400),
        (350.5, 500.4, 401, 500)
    ],
    'pm10': [
        (0, 54, 0, 50),
        (55, 154, 51, 100),
        (155, 254, 101, 150),
        (255, 354, 151, 200),
        (355, 424, 201, 300),
        (425, 504, 301, 400),
        (505, 604, 401, 500)
    ],
    'co': [
        (0.0, 4.4, 0, 50),
        (4.5, 9.4, 51, 100),
        (9.5, 12.4, 101, 150),
        (12.5, 15.4, 151, 200),
        (15.5, 30.4, 201, 300),
        (30.5, 40.4, 301, 400),
        (40.5, 50.4, 401, 500)
    ],
    'no2': [
        (0, 53, 0, 50),
        (54, 100, 51, 100),
        (101, 360, 101, 150),
        (361, 649, 151, 200),
        (650, 1249, 201, 300),
        (1250, 1649, 301, 400),
        (1650, 2049, 401, 500)
    ],
    'o3': [
        (0, 54, 0, 50),
        (55, 70, 51, 100),
        (71, 85, 101, 150),
        (86, 105, 151, 200),
        (106, 200, 201, 300),
        (201, 604, 301, 500)
    ]
}

def calculate_aqi(concentration, breakpoints):
    for (C_low, C_high, I_low, I_high) in breakpoints:
        if C_low <= concentration <= C_high:
            return (I_high - I_low) / (C_high - C_low) * (concentration - C_low) + I_low
    return None

# Calculate AQI for each pollutant
for pollutant in ['pm25', 'pm10', 'CO', 'NO2', 'O3']:
    merged_LA_df_pivot[f'AQI_{pollutant}'] = merged_LA_df_pivot[pollutant].apply(lambda x: calculate_aqi(x, aqi_breakpoints[pollutant]))

# Get the maximum AQI value for each row
aqi_columns = [f'AQI_{pollutant}' for pollutant in ['pm25', 'pm10', 'CO', 'NO2', 'O3']]
merged_LA_df_pivot['AQI'] = merged_LA_df_pivot[aqi_columns].max(axis=1)
